## 文本处理

In [1]:
with open('sentiment_dic/nagtive_dic_2.txt', 'r') as f:  # 替换成实际的文件名
    content = f.read().strip()  # 读取文件内容并去掉首尾空格
words = content.split('、')  # 将内容按顿号分割成单独的单词，存入列表中

with open('sentiment_dic/nagtive_dic_2.txt', 'w') as f:  # 替换成实际的文件名
    for word in words:
        f.write(word + '\n')  # 将每个单词写入文件，并换行

In [2]:
# 读取第一个txt文件
with open('sentiment_dic/nagtive_dic_1.txt', 'r') as f1:
    content1 = f1.read().strip()  # 读取文件内容并去掉首尾空格
words1 = content1.split('\n')  # 将内容按换行符分割成单独的单词，存入列表中

# 读取第二个txt文件
with open('sentiment_dic/nagtive_dic_2.txt', 'r') as f2:
    content2 = f2.read().strip()  # 读取文件内容并去掉首尾空格
words2 = content2.split('\n')  # 将内容按换行符分割成单独的单词，存入列表中

# 合并两个列表并去除重复的内容
words = list(set(words1 + words2))

# 将内容写入新的txt文件
with open('negative_dic_combined.txt', 'w') as f:
    for word in words:
        f.write(word + '\n')  # 将每个单词写入文件，并换行

## 情感值计算demo

In [111]:
import jieba

# 加载情感词典
with open('sentiment_dic/senti_dic_pro.txt', 'r', encoding='utf-8') as f:
    sentiment_dict = json.load(f)

# 加载程度副词词典
with open('sentiment_dic/adv_dic.txt', 'r', encoding='utf-8') as f:
    degree_dict = json.load(f)

# 加载否定词词典
with open('sentiment_dic/neg_dic.txt', 'r', encoding='utf-8') as f:
    negation_dict = json.load(f)
    
# 计算单个句子的情感分数
def sentiment_score(sen):
    seg_list = jieba.cut(sen)  # 分词
    seg_list = list(seg_list)
    print(seg_list)
    emotion_score = 0  # 情感分数
    neg_count = 0  # 否定词数量
    for i in range(len(seg_list)):
        # 检查情感词
        if seg_list[i] in sentiment_dict:
            score = sentiment_dict[seg_list[i]]
            degree_score = 1  # 默认程度系数为1
            negation_score = 1  # 默认否定系数为1
            # 在有效距离内寻找程度副词和否定词
            for j in range(max(0, i-3), i):
                if seg_list[j] in degree_dict:
                    degree_score *= degree_dict[seg_list[j]]
                elif seg_list[j] in negation_dict:
                    neg_count += 1
            # 根据否定词数量判断情感分数是否反转
            if neg_count % 2 == 0:
                emotion_score += score * degree_score
            else:
                emotion_score -= score * degree_score
            neg_count = 0  # 重置否定词数量
    return emotion_score

In [112]:
# 测试代码
sen = '我非常开心，甚至极其开心'
score = sentiment_score(sen)
print(score)

['我', '非常', '开心', '，', '甚至', '极其', '开心']
8


In [113]:
# 测试代码
sen = '很极其难过'
score = sentiment_score(sen)
print(score)

['很', '极其', '难过']
-6.0


In [114]:
# 测试代码
sen = '绝对很好'
score = sentiment_score(sen)
print(score)

['绝对', '很', '好']
6.0


In [115]:
# 测试代码
sen = '我好开心'
score = sentiment_score(sen)
print(score)

['我', '好开心']
0


In [116]:
# 测试代码
sen = '我挺好的'
score = sentiment_score(sen)
print(score)

['我', '挺', '好', '的']
1.6
